# A Priori Algorithm

In [1]:
SUPPORT_THRESHOLD = 1.9 # To avoid the mistake of using > instead of >=
CONFIDENCE_THRESHOLD = 0.599999

dataset = """HotDogs,	Buns,	Ketchup	
HotDogs,	Buns	
HotDogs,	Coke,	Chips	
Chips,	Coke	
Chips,	Ketchup	
HotDogs,	Coke,	Chips	""".replace("\t", "")

transactions = []
items = {} # dictionary of itesm and a list of  transactions they a p

# Problem would have been easier if I'd ised objects
from collections import namedtuple
Itemset = namedtuple('Itemset', 'items transactions hv support')

# Output for cell in CSV file
def symbol(transaction):
    return [item if item in transaction else '?' for item in items.keys()]
    

for i, line in enumerate(dataset.split("\n"),start=1):
    transaction = set(line.split(',')) # items in each line
    for item in transaction:
        l = items.get(item,[])
        l.append(str(i))
        items[item] = l
        
    transactions.append(transaction)
    
print "T", transactions
print "I", items
 
# Create CSV
print ",".join(items.keys())
for t in transactions:
    print ",".join(symbol(t))


T [set(['Buns', 'Ketchup', 'HotDogs']), set(['Buns', 'HotDogs']), set(['Coke', 'Chips', 'HotDogs']), set(['Coke', 'Chips']), set(['Chips', 'Ketchup']), set(['Coke', 'Chips', 'HotDogs'])]
I {'Buns': ['1', '2'], 'Chips': ['3', '4', '5', '6'], 'Ketchup': ['1', '5'], 'Coke': ['3', '4', '6'], 'HotDogs': ['1', '2', '3', '6']}
Buns,Chips,Ketchup,Coke,HotDogs
Buns,?,Ketchup,?,HotDogs
Buns,?,?,?,HotDogs
?,Chips,?,Coke,HotDogs
?,Chips,?,Coke,?
?,Chips,Ketchup,?,?
?,Chips,?,Coke,HotDogs


Create candidates, 

In [2]:
def join_items(A, B):
    # find the new items
    diff = A.items.symmetric_difference(B.items)
    
    # should be 2 new items e. {1,2,3,4} + {2,3,4,5}
    # {2,3,4} is common and {1,5} is difference
    if not len(diff) == 2: return None
    
    # find common transactions
    transactions = A.transactions & B.transactions
    
    # check the Itemset has enough support
    # this replaces pruning where we would have had to
    # check each proper subset of both A and B
    # we can do this either by carrying the transactions
    # or creating a dictionary but I chose this way
    # because from Big Data Management this would count
    if not len(transactions) >= SUPPORT_THRESHOLD: return None
    
    items = A.items | B.items
    
    # sort the items so that they can be hashed
    values = list(items)
    values.sort()
    
    return Itemset(items = items, transactions = transactions, hv = "".join(values), support=len(transactions) )

In [3]:
all_supported_itemsets = {}
current_itemsets = {}

# These will all be single itemsets
# item will be one item
# trans will be the transactions that item is in
for item, trans in items.items():
    if len(trans) >= SUPPORT_THRESHOLD:
        hv = "".join(item) # sets are unhashable, could make custom set
        support = len(trans) # is it better to store than call len(transactions)?
        itemset = Itemset(items = {item}, transactions = set(trans), hv=hv, support=support)
        current_itemsets[hv] = itemset
        print itemset
    
all_supported_itemsets.update(current_itemsets)

# Create comparison order
"""for a,b,c,d in current_itemsets.values():
    print a,b,c,d"""

# This is for FP
# values is all 1-itemsets
# ordered is a dictioanry of item, rank pairs
# so you can search for items by hash
# rather than look through a list
values = sorted(current_itemsets.values(), key=lambda itemset: -itemset.support)
ordered = {list(k.items)[-1] :v for v,k in enumerate(values)}

print ordered
        
while current_itemsets:
    from itertools import combinations
    combs = combinations(current_itemsets.values(), 2) # Self Join
    
    current_itemsets = {} # clear
    
    for c,d in combs: # for each candidate
        itemset = join_items(c,d)
        if itemset:   # rejected candidates will have returned None
            current_itemsets[itemset.hv] = itemset
            print "NEW ACCEPTED ITEMSET", itemset
        
    all_supported_itemsets.update(current_itemsets)

Itemset(items=set(['Buns']), transactions=set(['1', '2']), hv='Buns', support=2)
Itemset(items=set(['Chips']), transactions=set(['3', '5', '4', '6']), hv='Chips', support=4)
Itemset(items=set(['Ketchup']), transactions=set(['1', '5']), hv='Ketchup', support=2)
Itemset(items=set(['Coke']), transactions=set(['3', '4', '6']), hv='Coke', support=3)
Itemset(items=set(['HotDogs']), transactions=set(['1', '3', '2', '6']), hv='HotDogs', support=4)
{'Coke': 2, 'Ketchup': 4, 'Chips': 1, 'Buns': 3, 'HotDogs': 0}
NEW ACCEPTED ITEMSET Itemset(items=set(['Buns', 'HotDogs']), transactions=set(['1', '2']), hv='BunsHotDogs', support=2)
NEW ACCEPTED ITEMSET Itemset(items=set(['Chips', 'HotDogs']), transactions=set(['3', '6']), hv='ChipsHotDogs', support=2)
NEW ACCEPTED ITEMSET Itemset(items=set(['Coke', 'HotDogs']), transactions=set(['3', '6']), hv='CokeHotDogs', support=2)
NEW ACCEPTED ITEMSET Itemset(items=set(['Coke', 'Chips']), transactions=set(['3', '4', '6']), hv='ChipsCoke', support=3)
NEW ACCEPT

In [8]:
def write_line(ev, it):
    consequent = list(ev.items - it.items) # LOL this should be antecedend
    consequent.sort()
    confidence = ev.support*1.0/all_supported_itemsets["".join(consequent)].support
    support = len(ev.items)*1.0/it.support
    if confidence > CONFIDENCE_THRESHOLD:
        print ", ".join(consequent),"=> ", it.hv, "confidence =", confidence, "support =",support 

for ev in all_supported_itemsets.values():
    if len(ev.items) == 1:
        continue 
        
    for ite in ev.items:
        it = all_supported_itemsets[ite]
        write_line(ev,it)

Buns =>  HotDogs confidence = 1.0 support = 0.5
Chips =>  Coke confidence = 0.75 support = 0.666666666667
Coke =>  Chips confidence = 1.0 support = 0.5
Coke =>  HotDogs confidence = 0.666666666667 support = 0.5
Chips, HotDogs =>  Coke confidence = 1.0 support = 1.0
Coke, HotDogs =>  Chips confidence = 1.0 support = 0.75
Chips, Coke =>  HotDogs confidence = 0.666666666667 support = 0.75


# FP Growth

In [5]:
#from recordclass import recordclass

Node = namedtuple('Node', 'item support children')
head = Node(item=None, support={}, children = {}) 


for t in transactions:
    # Sort the items in each transaction
    ordered_transaction = sorted(t, key=lambda x: ordered[x]) 
    print ordered_transaction
    parent = head
    
    # Iterate through the items in each transaction
    # The first item will be a child of the head node
    # Then it will become parent so that
    # the next item adds itself as a child of
    # the preceding item in the transaction
    for item in ordered_transaction:
        
        child = parent.children.get(item, None)
        
        if child:
            child.support['s'] += 1
        else:
            new_item = Node(item = item, support = {'s':1}, children = {})
            parent.children[item] = new_item
            child = new_item
            
        parent = child

    print "\n\n\n\n", head


['HotDogs', 'Buns', 'Ketchup']




Node(item=None, support={}, children={'HotDogs': Node(item='HotDogs', support={'s': 1}, children={'Buns': Node(item='Buns', support={'s': 1}, children={'Ketchup': Node(item='Ketchup', support={'s': 1}, children={})})})})
['HotDogs', 'Buns']




Node(item=None, support={}, children={'HotDogs': Node(item='HotDogs', support={'s': 2}, children={'Buns': Node(item='Buns', support={'s': 2}, children={'Ketchup': Node(item='Ketchup', support={'s': 1}, children={})})})})
['HotDogs', 'Chips', 'Coke']




Node(item=None, support={}, children={'HotDogs': Node(item='HotDogs', support={'s': 3}, children={'Buns': Node(item='Buns', support={'s': 2}, children={'Ketchup': Node(item='Ketchup', support={'s': 1}, children={})}), 'Chips': Node(item='Chips', support={'s': 1}, children={'Coke': Node(item='Coke', support={'s': 1}, children={})})})})
['Chips', 'Coke']




Node(item=None, support={}, children={'Chips': Node(item='Chips', support={'s': 1}, children={'Coke': Node(

In [6]:
from graphviz import Digraph

# Was going to visualize this but graphs need unique node names
# So became more trouble than worth
dot = Digraph(comment='FP Tree')

#Depth-First Traversal
nodes = []

for n in head.children.values():
        nodes.append([n])

while nodes:
    node_list = nodes.pop()
    print "-> ".join([x.item for x in node_list]), "Support =", node_list[-1].support['s']
    node = node_list[-1]
    for n in node.children.values():
        nodes.append(node_list + [n])
        

HotDogs Support = 4
HotDogs-> Chips Support = 2
HotDogs-> Chips-> Coke Support = 2
HotDogs-> Buns Support = 2
HotDogs-> Buns-> Ketchup Support = 1
Chips Support = 2
Chips-> Ketchup Support = 1
Chips-> Coke Support = 1


In [7]:
n = [1,2] + [3]
print n

[1, 2, 3]
